In [7]:
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.21.0
    Uninstalling google-cloud-bigquery-3.21.0:
      Successfully uninstalled google-cloud-bigquery-3.21.0


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in resolve
    self._add_to_criteria(self.state.criteria, r, parent=None)
  File "/usr/local/lib/pytho

In [5]:
import base64
import pandas as pd
import vertexai
from google.colab import auth
auth.authenticate_user()
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Load your database into a Pandas dataframe
df = pd.read_csv("hf://datasets/Kaludi/Customer-Support-Responses/Customer-Support.csv")

def get_ask(question):
    while True:
        answer = input(question + " (Y/N): ")
        if answer.lower() == 'y':
            return True
        elif answer.lower() == 'n':
            return False
        else:
            print("Please enter Y or N")

def multiturn_generate_content():
    vertexai.init(project="bubbly-granite-418011", location="asia-south1")
    model = GenerativeModel(
        "gemini-1.5-pro-001",
        system_instruction=[
            "You are a customer service agent",
            *["{}: {}".format(query, response) for query, response in zip(df['query'], df['response'])]
        ]
    )
    chat = model.start_chat()

    while True:
        user_input = input("Please enter your query: ")
        print(chat.send_message(
            [user_input],
            generation_config=generation_config,
            safety_settings=safety_settings
        ))

        if not get_ask("Do you want to ask another question"):
            break

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

multiturn_generate_content()

Please enter your query: I received damaged product
candidates {
  content {
    role: "model"
    parts {
      text: "We apologize for the inconvenience of receiving a damaged product. To help you as quickly as possible, could you please provide a photo of the damaged product? This will allow us to assess the damage and determine the best course of action. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.07509511709213257
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.058219537138938904
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.056587908416986465
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.018520791083574295
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.15133880078792572
    severity: HARM_SEVERITY_NEGLIGIBLE
